# 기본 설정

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS as stopwords
from PIL import Image, ImageFilter
from wordcloud import ImageColorGenerator
from matplotlib import font_manager, rc
from collections import Counter
import konlpy
from konlpy.tag import Mecab
import time
from random import shuffle
import json
from tqdm import tqdm
tqdm.pandas()
import gensim 
from gensim.models import Word2Vec, FastText, Doc2Vec
import fasttext
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Loading BokehJS ...

In [2]:
bills = pd.read_csv('1차 데이터 및 jupyter/bills_24839_all_cleansed.csv')
senators = pd.read_csv('1차 데이터 및 jupyter/(원본) assembly_memebers_295.csv')

In [3]:
senators['한글이름'] = senators['이름'].str.split(' ').str[0]
senators['한자이름'] = senators['이름'].str.split(' ').str[1]

In [4]:
bills = bills[bills['발의자'] != '위원장']
bills = bills[bills['발의자'] != '정부'] 
bills = bills.dropna()

# 유저 인풋 만들기

In [5]:
embedding_model = Word2Vec.load('models/word2vec ver21.0.model')

In [6]:
with open('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/senators ver21.0.json', 'r') as f:
    senators_json = json.load(f)

In [7]:
word_dict = {}
for vocab, vector in zip(embedding_model.wv.index2word, embedding_model.wv.vectors):
    word_dict[vocab] = vector

### 함수 모음

In [8]:
def search(user_input):
    temp_list = []
    for word in word_dict:
        if user_input in word:
            temp_list.append(word)
    return temp_list

def similarWord(model, text):
    return model.most_similar(positive=[text], topn=30)

def similar_senators(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    return rating[:5]

def show_senators_bills_contents(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_ids = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['bill_id'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_ids.extend(semi_result)
        list_temp_keyword_bills_ids = list(set(list_temp_keyword_bills_ids))
        shuffle(list_temp_keyword_bills_ids)
        list_bills_title = df_bills[df_bills['bill_id'].isin(list_temp_keyword_bills_ids[:3])]['법안명'].values.tolist()
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

def show_senators_bills_titles(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_titles = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['법안명'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_titles.extend(semi_result)
        list_temp_keyword_bills_titles = list(set(list_temp_keyword_bills_titles))
        shuffle(list_temp_keyword_bills_titles)
        print(list_temp_keyword_bills_titles)
        list_bills_titlle = df_bills.loc(list_temp_keyword_bills_titles[:3])
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

## 시연

In [9]:
search('청년')

['청년', '청년정책', '청년층', '청년실업률', '청년기본소득']

In [10]:
similarWord(embedding_model, '청년')

<ipython-input-8-a19bf89c2456>:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return model.most_similar(positive=[text], topn=30)


[('미취', 0.7219280004501343),
 ('고용촉진', 0.6167274713516235),
 ('실업', 0.5831162929534912),
 ('청년정책', 0.5714389681816101),
 ('일자리', 0.5657309293746948),
 ('창업', 0.5624954700469971),
 ('청년층', 0.5527931451797485),
 ('비정규직', 0.5297333002090454),
 ('고용', 0.5255996584892273),
 ('구직', 0.5193198323249817),
 ('취업', 0.5191558599472046),
 ('정규직', 0.4891175627708435),
 ('중소기업', 0.4889249801635742),
 ('저출산', 0.4814630150794983),
 ('사회상', 0.4796830415725708),
 ('실업자', 0.47934767603874207),
 ('채움', 0.47375449538230896),
 ('신혼', 0.4728226065635681),
 ('지방공기업', 0.4708821475505829),
 ('내일', 0.4685583710670471),
 ('주거지원', 0.4595612585544586),
 ('고령사회', 0.4569365680217743),
 ('인재', 0.4538780748844147),
 ('창업자', 0.4537651538848877),
 ('직업', 0.45215409994125366),
 ('농업인', 0.44717732071876526),
 ('창출', 0.44420331716537476),
 ('저성장', 0.44353532791137695),
 ('예술인', 0.4422413110733032),
 ('고용창출', 0.4418375790119171)]

In [116]:
user_input=['취업', '비정규직', '근로조건','청년실업률','취업지원']

In [128]:
user_input=['복지서비스','장애인', '특수교육','장애인복지시설']

In [129]:
list_senator = similar_senators(user_input, senators_json, word_dict)
list_senator

['여영국 정의당', '오영훈 더불어민주당', '정춘숙 더불어민주당', '유승민 미래통합당', '나경원 미래통합당']

In [130]:
show_senators_bills_contents(list_senator, bills)

{'여영국 정의당': ['사회복지사 등의 처우 및 지위 향상을 위한 법률 일부개정법률안(정동영의원 등 10인)',
  '국회법 일부개정법률안(추혜선의원 등 14인)',
  '장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(추혜선의원 등 10인)'],
 '오영훈 더불어민주당': ['정신건강증진 및 정신질환자 복지서비스 지원에 관한 법률 일부개정법률안(신동근의원 등 11인)',
  '장애인·노인·임산부 등의 편의증진 보장에 관한 법률 일부개정법률안(김병욱의원 등 12인)',
  '노숙인 등의 복지 및 자립지원에 관한 법률 일부개정법률안(조승래의원 등 10인)'],
 '정춘숙 더불어민주당': ['다중이용업소의 안전관리에 관한 특별법 일부개정법률안(신창현의원 등 15인)',
  '수상레저안전법 일부개정법률안(양승조의원 등 10인)',
  '아동복지법 일부개정법률안(김상희의원 등 10인)'],
 '유승민 미래통합당': ['참전유공자 예우 및 단체설립에 관한 법률 일부개정법률안(지상욱의원 등 11인)',
  '국가유공자 등 예우 및 지원에 관한 법률 일부개정법률안(지상욱의원 등 11인)',
  '장애인·고령자 등 주거약자 지원에 관한 법률 일부개정법률안(김현아의원 등 10인)'],
 '나경원 미래통합당': ['장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(이주영의원 등 10인)',
  '장기공공임대주택 입주자 삶의 질 향상 지원법 일부개정법률안(김성태의원 등 10인)',
  '재난 및 안전관리 기본법 일부개정법률안(이종명의원 등 13인)']}

In [119]:
show_senators_bills_contents(list_senator, bills)

{'김현미 더불어민주당': ['취업 후 학자금 상환 특별법 일부개정법률안(김민기의원 등 23인)',
  '조세특례제한법 일부개정법률안(박광온의원 등 10인)',
  '조세특례제한법 일부개정법률안(윤호중의원 등 10인)'],
 '심상정 정의당': ['공직자의 이해충돌 방지 및 윤리에 관한 법률안(심상정의원 등 10인)',
  '성별에 의한 차별ㆍ성희롱 금지 및 권리구제 등에 관한 법률안(남인순의원 등 10인)',
  '보건의료 인력지원 특별법안(정춘숙의원 등 11인)'],
 '이정미 정의당': ['근로기준법 일부개정법률안(이용득의원 등 10인)',
  '아동·청소년의 성보호에 관한 법률 일부개정법률안(김상희의원 등 13인)',
  '국가재정법 일부개정법률안(정세균의원 등 108인)'],
 '김부겸 더불어민주당': ['생활체육진흥법 일부개정법률안(김부겸의원 등 18인)',
  '청년고용촉진 특별법 일부개정법률안(이용득의원 등 37인)',
  '노동조합 및 노동관계조정법 일부개정법률안(송옥주의원 등 20인)'],
 '김태년 더불어민주당': ['고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 일부개정법률안(한정애의원 등 16인)',
  '지방공무원법 일부개정법률안(박광온의원 등 10인)',
  '청년세법안(정세균의원 등 108인)']}